In [1]:
import os

In [2]:
from modules.pipeline import *

modules.report successfull loaded: 100%|██████████| 6/6 [00:19<00:00,  3.28s/it]


In [4]:
from unidecode import unidecode

In [5]:
import re

In [6]:
from copy import deepcopy

In [7]:
def get_filenames(root_dir, func, verbose=False):
    data_filenames = []
    # Walk through the directories and files
    for dirpath, dirnames, filenames in os.walk(root_dir):
        # dirpath: current directory path
        # dirnames: list of directories in the current directory
        # filenames: list of files in the current directory

        # Print the current directory
        print('Directory:', dirpath)  if verbose else None
        # Print all the subdirectories
        if verbose:
            for dirname in dirnames:
                print('Subdirectory:', os.path.join(dirpath, dirname))

        # Print all the files
        for filename in filenames:
            if func(filename) and not ('x_' in filename or 'y_' in filename or 'metric' in filename):
                print('File:', os.path.join(dirpath, filename)) if verbose else None
                data_filenames.append(os.path.join(dirpath, filename))

        # Print an empty line to separate directories
        print()  if verbose else None
    return data_filenames

In [8]:
MLN_F = lambda x: (('classic_mln_' in x) and not('classic_mln_-' in x)) # find metric of model where mln were added
MLN__F = lambda x: (('classic_mln_-' in x)) # where mln attribut were removed first
MLN_C_F = lambda x: (('classic_-' in x)) # where mln attribut were removed first
MLN_C= lambda x: (('classic_' in x)) # where mln attribut were removed first

INTER_F = lambda x: (not('_max_' in x) and ('inter' in x))
INTRA_F = lambda x: (not('_max_' in x) and ('intra' in x))
COMBINE_F = lambda x: (not('_max_' in x) and ('combine' in x))
ULTRA_F = lambda x: (not('_max_' in x) and ('ultra' in x))
INTER_MAX_F = lambda x: (('_max_' in x) and ('inter' in x))
INTRA_MAX_F = lambda x: (('_max_' in x) and ('intra' in x))
COMBINE_MAX_F = lambda x: (('_max_' in x) and ('combine' in x))
ULTRA_MAX_F = lambda x: (('_max_' in x) and ('ultra' in x))
DEGREE_F = lambda x: (('degree' in x))

In [8]:
def metrics_analyzer_plots(cols=None, outputs_path=None, cwd=None, data_folder=None, classic_metrics=None, models_name=None):
    """ build relevance results about the datasset
    
    Args:
        - cols: list of qualitative variable in the dataset
        - outputs_path: the path where the experimental results are located
        - 
    
    Returns:
        A dedicated folder with those relevante reports and charts
    
    """
    outputs = {}
    
    if cols != None or classic_metrics != None: # check if cols and classics metrics are filled
        ## analyse of k layer
        for k in list(set([1, 2, len(cols)])):
            #for logic in ['global', 'personalized']:
            #outputs[logic] = {}
            ### get all combination of col
            for layer_config in get_combinations(range(len(cols)),k): # create subsets of k index of OHE and fetch it
                col_targeted= [f'{cols[i]}' for i in layer_config]
                case_k= '±'.join(col_targeted) if len(layer_config)>1 else col_targeted[0]
                if sum(
                        [
                            re.sub(r'[^\w\s]', '', unidecode(partern)) in re.sub(r'[^\w\s]', '', unidecode('cb_person_default_on_file±loan_intent'))
                            for partern in case_k.split("±")
                            ]
                        ) == k: # check mission context
                    continue
                print(case_k)
                
                ### get files for distincts logic
                match= lambda x: (
                    sum(
                        [
                            re.sub(r'[^\w\s]', '', unidecode(partern)) in re.sub(r'[^\w\s]', '', unidecode(x))
                            for partern in case_k.split("±")
                            ]
                        ) == k if k > 1 else re.sub(r'[^\w\s]', '', unidecode(case_k)) in re.sub(r'[^\w\s]', '', unidecode(x))
                    )
                files = {
                    'global':{
                        'classic': classic_metrics,
                        'classic_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                                func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    },
                    "personalized":{
                        'classic': classic_metrics,
                        'classic_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                                func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    }
                }
                # print(files)
                #outputs[logic] = files
                ### transform and normalize
                models_list = files['personalized']['classic'].index.values.tolist()
                print(models_list)
                metrics = ["accuracy","precision","recall","f1-score"]
                models = {}
                models_1 = {}
                for model in models_list:
                    data = {
                        'global':{},
                        'personalized':{}
                    }
                    for key in files['global'].keys():
                        data['global'][key] = [files['global'][key].loc[model,metric] for metric in metrics]
                        data['personalized'][key] = [files['personalized'][key].loc[model,metric] for metric in metrics]

                    data['global']['Metrics'] = metrics
                    data['personalized']['Metrics'] = metrics
                    models[model] = pd.DataFrame(data=data['global'])
                    models_1[model] = pd.DataFrame(data=data['personalized'])
                ### generate figures
                nrow = 2
                ncol = 4
                
                fig, axs = plt.subplots(nrows=nrow, ncols=ncol, figsize=(20, 15))
                count=0
                for r in range(ncol): 
                    # Barplot i
                    models[models_list[count]].plot(kind='bar', x='Metrics', y=files['global'].keys(), ax=axs[0,r])
                    axs[0,r].set_title(f'{models_name[models_list[count]]}')
                    axs[0,r].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
                    
                    models_1[models_list[count]].plot(kind='bar', x='Metrics', y=files['personalized'].keys(), ax=axs[1,r])
                    axs[1,r].set_title(f'{models_name[models_list[count]]}')
                    axs[1,r].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
                    
                    axs[0,r].legend().set_visible(False)
                    axs[1,r].legend().set_visible(False)
                    if r == ncol-1:
                        axs[1,r].legend().set_visible(True)
                    count+=1
                axs[0,0].set_ylabel('global')
                axs[1,0].set_ylabel('personalized')
                if True:
                    create_domain(f'{cwd}/analyser/{data_folder}/plots/mlna_{k}/mixed')

                    timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
                    filename1 = f'{cwd}/analyser/{data_folder}/plots/mlna_{k}/mixed/_metrics_comparaison_for_{case_k}'+'_'+timestr+'.png'
                    # Adjust the layout to cover all content
                    plt.tight_layout()

                    plt.savefig(filename1,dpi=150) #.png,.pdf will also support here
                    plt.close() # close the plot windows
                ### generate reports
    return outputs



In [9]:
def metrics_analyzer_reports(cols=None, outputs_path=None, cwd=None, data_folder=None, classic_metrics=None, models_name=None):
    """ build relevance results about the datasset
    
    Args:
        - cols: list of qualitative variable in the dataset
        - outputs_path: the path where the experimental results are located
        - 
    
    Returns:
        A dedicated folder with those relevante reports and charts
    
    """
    day = time.strftime("%Y_%m_%d_%H")
    if cols != None or classic_metrics != None: # check if cols and classics metrics are filled
        ## analyse of k layer
        head_lambda = lambda x: f'<tr><td rowspan="2" colspan="3">{x}</td><td colspan="4">Classic</td><td colspan="4">Classic - Att</td><td colspan="4">Classic + mln</td><td colspan="4">Classic + mln - Att</td></tr><tr class="metrics">{"<td>Accuracy</td><td>Precision</td><td>Recall</td><td>F1-score</td>" * 4}</tr>'
        head = {
            key: head_lambda(key)
            for key in models_name.keys()
        }
        body = {
            key: ''
            for key in models_name.keys()
        }
        style = '<style>tr:nth-child(even) {background-color: #dddddd;} table, th, td {border: 1px solid black;border-collapse: collapse;} .wrap-text {word-wrap: break-word;} .wrap-text {overflow-wrap: break-word;} .limited-column {width: 100px;} .dashed-border {border: 1px dashed black;}.dotted-border {border: 1px dotted black;} td {text-align: center;} caption {margin:0; margin-bottom: 2px; text-align: start; border: 1px dashed black;} caption > h2 {text-align: center;}</style>'
        
        caption_content_lambda = lambda x: ''.join([f'<span><strong>{key}</strong>: {value}</span><br>' for key, value in {
            f'{x}': models_name[x],
            'MLN': 'MultiLayer Network',
            'MLN k Layer(s)': 'MLN with k layer(s)',
            'Att': 'Attributs or modalities of variable(s) used to build MLN',
            'mln': 'Descriptors extracted from MLN',
            'Classic': f'Learning from classic dataset of {data_folder}',
            'Classic - Att': f'Learning from classic dataset of {data_folder} where Att had been removed',
            'Classic + mln': f'Learning from classic dataset of {data_folder} where mln had been added',
            'Classic + mln - Att': f'Learning from classic dataset of {data_folder} where mln had been added and Att removed'
            }.items()])

        for k in list(set([1, 2, len(cols)])):
            #for logic in ['global', 'personalized']:
            #outputs[logic] = {}
            ### get all combination of col
            body_l = {
                key: ''
                for key in models_name.keys()
            }
            LayerLines = f'<tr style="border-top: 2px solid black;"><td rowspan="{len(get_combinations(range(len(cols)),k)) * 2}" >MLN {k} layer(s)</td>'
            
            for layer_config in get_combinations(range(len(cols)),k): # create subsets of k index of OHE and fetch it
                col_targeted= [f'{cols[i]}' for i in layer_config]
                case_k= '±'.join(col_targeted) if len(layer_config)>1 else col_targeted[0]
                #if sum(
                #        [
                #            re["s"]ub(r'[^\w\s]', '', unidecode(partern)) in re["s"]ub(r'[^\w\s]', '', unidecode('cb_person_default_on_file±loan_intent'))
                #            for partern in case_k["s"]plit("±")
                #            ]
                #        ) == k: # check mission context
                #    continue
                print(case_k)
                
                VarLines = f'<td rowspan="2" >{case_k if k < len(cols) else "All"}</td>'
                
                ### get files for distincts logic
                match= lambda x: (
                    sum(
                        [
                            re.sub(r'[^\w\s]', '', unidecode(partern)) in re.sub(r'[^\w\s]', '', unidecode(x))
                            for partern in case_k.split("±")
                            ]
                        ) == k if k > 1 else re.sub(r'[^\w\s]', '', unidecode(case_k)) in re.sub(r'[^\w\s]', '', unidecode(x))
                    )
                files = {
                    'global':{
                        'classic': classic_metrics,
                        'classic_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                                func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    },
                    "personalized":{
                        'classic': classic_metrics,
                        'classic_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                                func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    }
                }
                # print(files)
                #outputs[logic] = files
                ### transform and normalize
                models_list = files['personalized']['classic'].index.values.tolist()
                print(models_list)
                metrics = ["accuracy","precision","recall","f1-score"]
                
                for model in models_list:
                    max_g = {metric:
                        max([round(files['global'][key].loc[model,metric],4) for key in files['global'].keys()]) for metric in metrics
                        }
                    max_p = {metric:
                        max([round(files['personalized'][key].loc[model,metric],4) for key in files['personalized'].keys()]) for metric in metrics
                        }
                    data = {
                        'global':'<td>Global</td>',
                        'personalized':f'<tr><td>Personalized</td>'
                    }
                    for key in files['global'].keys():
                        data['global']+= ''.join([ 
                            f'<td style={"color:blue; font-size: 40px; font-weight: bold;" * int(round(files["global"][key].loc[model,metric],4) == max_g[metric])}>{round(files["global"][key].loc[model,metric],4)}</td>' 
                            for metric in metrics
                            ])
                        data['personalized']+= ''.join([ 
                            f'<td style={"color:blue; font-size: 40px; font-weight: bold;" * int(round(files["personalized"][key].loc[model,metric],4) == max_p[metric])}>{round(files["personalized"][key].loc[model,metric],4)}</td>' 
                            for metric in metrics
                            ])
                    data['global']+= '</tr>'
                    data['personalized']+= '</tr>'
                    body_l[model] += (LayerLines + VarLines + data['global'] + data['personalized'])  if len(body_l[model]) == 0 else ( f'<tr >' + VarLines + data['global'] + data['personalized'])
            
            for model in models_name.keys():
                body[model]+= body_l[model]
                caption = f'<caption><h2>Legend</h2>{caption_content_lambda(model)}</caption>'
                table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{caption}{head[model]}{body_l[model]}</table>'
                htm = f'<html><head>{style}<title> Summary </title></head><body style="background-color: white;">{table_html}</body></html>'

                create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/mlna_{k}/report')
                timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
                filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/mlna_{k}/report/Summary {data_folder} for {model} '+'_'+timestr+'.html'
                _file= open(filename1,"w")
                _file.write(htm)
                _file.close()
        for model in models_name.keys():
            caption = f'<caption><h2>Legend</h2>{caption_content_lambda(model)}</caption>'
            table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{caption}{head[model]}{body[model]}</table>'
            htm = f'<html><head>{style}<title> Summary </title></head><body style="background-color: white;">{table_html}</body></html>'
            
            create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/report')
            timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
            filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/report/Summary {data_folder} for {model} '+'_'+timestr+'.html'
            _file= open(filename1,"w")
            _file.write(htm)
            _file.close()



In [10]:
def metrics_analyzer_statistics(cols=None, outputs_path=None, cwd=None, data_folder=None, classic_metrics=None, models_name=None):
    """ build relevance results about the datasset
    
    Args:
        - cols: list of qualitative variable in the dataset
        - outputs_path: the path where the experimental results are located
        - 
    
    Returns:
        A dedicated folder with those relevante reports and charts
    
    """
    day = time.strftime("%Y_%m_%d_%H")
    if cols != None or classic_metrics != None: # check if cols and classics metrics are filled
        ## analyse of k layer

        # find out all best metric details
        head_lambda = lambda x: f'<tr><td colspan="2">{x}</td><td>Classic</td><td>Best {x}</td><td>Up (%)</td><td>Approach(s)</td><td>Logic(s)</td></tr>'
        tab1_head = {
            'accuracy': head_lambda('accuracy'),
            'precision': head_lambda('precision'),
            'recall': head_lambda('recall'),
            'f1-score': head_lambda('f1-score')
        }
        tab1_body = {
            'accuracy': '',
            'precision': '',
            'recall': '',
            'f1-score': ''
        }

        # analyse the impact of each approach on classic
        head_lambda = lambda x: f'<tr><td colspan="2" rowspan="2">{x}</td><td colspan="3">Classic - Att Vs Classic</td><td colspan="3">Classic + MLN Vs Classic</td><td colspan="3">Classic + MLN - Att Vs Classic</td></tr><tr>{"<td>Classic ></td><td>Classic =</td><td>Classic <</td>"*3}</tr>'
        tab2_head_g = {
            'accuracy': head_lambda('accuracy'),
            'precision': head_lambda('precision'),
            'recall': head_lambda('recall'),
            'f1-score': head_lambda('f1-score')
        }
        tab2_head_p = deepcopy(tab2_head_g)
        tab2_body_g = {
            'accuracy': '',
            'precision': '',
            'recall': '',
            'f1-score': ''
        }
        tab2_body_p = deepcopy(tab2_body_g)

        # check if corelation between class of Att in MLN1 and their classe in MLN 2+
        head_lambda = lambda x: f'<tr><td colspan="2" rowspan="2">{x}</td><td colspan="3">(Good MLN 1, Good MLN 1)</td><td colspan="3">(Good MLN 1, Bad MLN 1)</td><td colspan="3">(Bad MLN 1, Bad MLN 1)</td></tr><tr>{"<td>Classic ></td><td>Classic =</td><td>Classic <</td>"*3}</tr>'
        tab3_head_g = {
            'accuracy': head_lambda('accuracy'),
            'precision': head_lambda('precision'),
            'recall': head_lambda('recall'),
            'f1-score': head_lambda('f1-score')
        }
        tab3_head_p = deepcopy(tab3_head_g)
        tab3_body_g1 = {
            'accuracy': '',
            'precision': '',
            'recall': '',
            'f1-score': ''
        }
        tab3_body_g2 = deepcopy(tab3_body_g1)
        tab3_body_p1 = deepcopy(tab3_body_g1)
        tab3_body_p2 = deepcopy(tab3_body_p1)

        # compare global and personalized logics
        head_lambda = lambda x: f'<tr><td colspan="2" rowspan="2">{x}</td><td colspan="3">Classic + MLN</td><td colspan="3">Classic + MLN - Att</td></tr><tr>{"<td>perso ></td><td>perso =</td><td>perso <</td>"*2}</tr>'
        tab4_head = {
            'accuracy': head_lambda('accuracy'),
            'precision': head_lambda('precision'),
            'recall': head_lambda('recall'),
            'f1-score': head_lambda('f1-score')
        }
        tab4_body = {
            'accuracy': '',
            'precision': '',
            'recall': '',
            'f1-score': ''
        }
        
        style = '<style> table, th, td {border: 1px solid black;border-collapse: collapse;} .wrap-text {word-wrap: break-word;} .wrap-text {overflow-wrap: break-word;} .limited-column {width: 100px;} .dashed-border {border: 1px dashed black;}.dotted-border {border: 1px dotted black;} td {text-align: center;} caption {margin:0; margin-bottom: 2px; text-align: start; border: 1px dashed black;} caption > h2 {text-align: center;}</style>'
        
        caption_content_lambda = lambda x: ''.join([f'<span><strong>{key}</strong>: {value}</span><br>' for key, value in {
            **models_name,
            'MLN k Layer(s)': 'MultiLayer Network with k layer(s)',
            'Att': 'Attributs or modalities of variable(s) used to build MLN',
            'MLN': 'Descriptors extracted from MLN',
            'Classic': f'Learning from classic dataset of {data_folder}',
            'Classic - Att': f'Learning from classic dataset of {data_folder} where Att had been removed',
            'Classic + MLN': f'Learning from classic dataset of {data_folder} where MLN had been added',
            'Classic + MLN - Att': f'Learning from classic dataset of {data_folder} where MLN had been added and Att removed'
            }.items()])
        # save class of each Att
        clusters = {
            'accuracy': {
                'Good': [],
                'Bad': []
            },
            'precision': {
                'Good': [],
                'Bad': []
            },
            'recall': {
                'Good': [],
                'Bad': []
            },
            'f1-score': {
                'Good': [],
                'Bad': []
            }
        }
        # base format for storing best metrics
        best_metrics = {
            'accuracy': {
                'value': 0,
                'approachs':[],
                'logics':[]
            },
            'precision': {
                'value': 0,
                'approachs':[],
                'logics':[]
            },
            'recall': {
                'value': 0,
                'approachs':[],
                'logics':[]
            },
            'f1-score': {
                'value': 0,
                'approachs':[],
                'logics':[]
            }
        }
        comp1 = {
            'accuracy': [{
                's': 0,
                'e':0,
                'i':0
            },{
                's': 0,
                'e':0,
                'i':0
            },{
                's': 0,
                'e':0,
                'i':0
            }],
            'precision': [{
                's': 0,
                'e':0,
                'i':0
            },{
                's': 0,
                'e':0,
                'i':0
            },{
                's': 0,
                'e':0,
                'i':0
            }],
            'recall': [{
                's': 0,
                'e':0,
                'i':0
            },{
                's': 0,
                'e':0,
                'i':0
            },{
                's': 0,
                'e':0,
                'i':0
            }],
            'f1-score': [{
                's': 0,
                'e':0,
                'i':0
            },{
                's': 0,
                'e':0,
                'i':0
            },{
                's': 0,
                'e':0,
                'i':0
            }]
        }
        comp2 = {
            'accuracy': [{
                's': 0,
                'e':0,
                'i':0
            },{
                's': 0,
                'e':0,
                'i':0
            }],
            'precision': [{
                's': 0,
                'e':0,
                'i':0
            },{
                's': 0,
                'e':0,
                'i':0
            }],
            'recall': [{
                's': 0,
                'e':0,
                'i':0
            },{
                's': 0,
                'e':0,
                'i':0
            }],
            'f1-score': [{
                's': 0,
                'e':0,
                'i':0
            },{
                's': 0,
                'e':0,
                'i':0
            }]
        }
        clusters_g1 = {key: deepcopy(clusters) for key in models_name.keys()}
        clusters_g2 = {key: deepcopy(clusters) for key in models_name.keys()}
        clusters_p1 = {key: deepcopy(clusters) for key in models_name.keys()}
        clusters_p2 = {key: deepcopy(clusters) for key in models_name.keys()}
        # save for each model, this best metric
        body_best = {
            key: deepcopy(best_metrics) for key in models_name.keys()
        }
        for k in list(set([1, 2, len(cols)])):
            #for logic in ['global', 'personalized']:
            #outputs[logic] = {}
            ### get all combination of col

            body_compA_g = {
                key: deepcopy(comp1) for key in models_name.keys()
            }
            body_compA_p = deepcopy(body_compA_g)
            body_compC_g1 = {
                key: deepcopy(comp1) for key in models_name.keys()
            }
            body_compC_g2 = deepcopy(body_compC_g1)
            body_compC_p1 = deepcopy(body_compC_g1)
            body_compC_p2 = deepcopy(body_compC_g1)
            body_compL = {
                key: deepcopy(comp2) for key in models_name.keys()
            }
            
            # aside row config
            LayerLines = f'<tr style="border-top: 2px solid black;"><td rowspan="{len(models_name.keys()) }" >MLN {k} layer(s): {len(get_combinations(range(len(cols)),k))} Att</td>'
            
            for layer_config in get_combinations(range(len(cols)),k): # create subsets of k index of OHE and fetch it
                col_targeted= [f'{cols[i]}' for i in layer_config]
                case_k= '±'.join(col_targeted) if len(layer_config)>1 else col_targeted[0]
                
                ### get files for distincts logic
                match= lambda x: (
                    sum(
                        [
                            re.sub(r'[^\w\s]', '', unidecode(partern)) in re.sub(r'[^\w\s]', '', unidecode(x))
                            for partern in case_k.split("±")
                            ]
                        ) == k if k > 1 else re.sub(r'[^\w\s]', '', unidecode(case_k)) in re.sub(r'[^\w\s]', '', unidecode(x))
                    )
                files = {
                    'global':{
                        'classic': classic_metrics,
                        'classic_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                                func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/global/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    },
                    "personalized":{
                        'classic': classic_metrics,
                        'classic_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/data_selection_storage', 
                                func=lambda x: ((MLN_C_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN_F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1],
                        'classic_mln_-_mlna':[
                            load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                            for file in get_filenames(
                                root_dir=f'{outputs_path}/qualitative/mlna_{k}/personalized/data_selection_storage', 
                                func=lambda x: ((MLN__F(x)) and (match(x))), 
                                verbose=False
                                )
                            ][-1]
                    }
                }
                # print(files)
                #outputs[logic] = files
                ### transform and normalize
                models_list = files['personalized']['classic'].index.values.tolist()
                print(models_list)
                metrics = ["accuracy","precision","recall","f1-score"]
                
                for model in models_list:
                    #print(body_best, '\n----------------------')
                    for metric in metrics:
                        if k == 1: # in 1st layer, find Att which increase or not metrics
                            clusters_g1[model][metric]['Good' if round(files['global']['classic_mln'].loc[model,metric],4) > round(files['global']['classic'].loc[model,metric],4) else 'Bad'].append(case_k)
                            clusters_g2[model][metric]['Good' if round(files['global']['classic_mln_-_mlna'].loc[model,metric],4) > round(files['global']['classic'].loc[model,metric],4) else 'Bad'].append(case_k)
                            clusters_p1[model][metric]['Good' if round(files['personalized']['classic_mln'].loc[model,metric],4) > round(files['personalized']['classic'].loc[model,metric],4) else 'Bad'].append(case_k)
                            clusters_p2[model][metric]['Good' if round(files['personalized']['classic_mln_-_mlna'].loc[model,metric],4) > round(files['personalized']['classic'].loc[model,metric],4) else 'Bad'].append(case_k)
                        # count combination impact
                        if k == 2:
                            if sum([partern in clusters_g1[model][metric]['Good'] for partern in case_k.split("±")]) == k:
                               indice = 0
                            elif sum([partern in clusters_g1[model][metric]['Bad'] for partern in case_k.split("±")]) == k:
                               indice = 2
                            else:
                               indice = 1
                                
                            body_compC_g1[model][metric][indice]["s"] += 1 if round(files['global']['classic_mln'].loc[model,metric],4) < round(files['global']['classic'].loc[model,metric],4) else 0
                            body_compC_g1[model][metric][indice]["e"] += 1 if round(files['global']['classic_mln'].loc[model,metric],4) == round(files['global']['classic'].loc[model,metric],4) else 0
                            body_compC_g1[model][metric][indice]["i"] += 1 if round(files['global']['classic_mln'].loc[model,metric],4) > round(files['global']['classic'].loc[model,metric],4) else 0

                            if sum([partern in clusters_g2[model][metric]['Good'] for partern in case_k.split("±")]) == k:
                               indice = 0
                            elif sum([partern in clusters_g2[model][metric]['Bad'] for partern in case_k.split("±")]) == k:
                               indice = 2
                            else:
                               indice = 1
                                
                            body_compC_g2[model][metric][indice]["s"] += 1 if round(files['global']['classic_mln_-_mlna'].loc[model,metric],4) < round(files['global']['classic'].loc[model,metric],4) else 0
                            body_compC_g2[model][metric][indice]["e"] += 1 if round(files['global']['classic_mln_-_mlna'].loc[model,metric],4) == round(files['global']['classic'].loc[model,metric],4) else 0
                            body_compC_g2[model][metric][indice]["i"] += 1 if round(files['global']['classic_mln_-_mlna'].loc[model,metric],4) > round(files['global']['classic'].loc[model,metric],4) else 0

                            if sum([partern in clusters_p1[model][metric]['Good'] for partern in case_k.split("±")]) == k:
                               indice = 0
                            elif sum([partern in clusters_p1[model][metric]['Bad'] for partern in case_k.split("±")]) == k:
                               indice = 2
                            else:
                               indice = 1
                                
                            body_compC_p1[model][metric][indice]["s"] += 1 if round(files['personalized']['classic_mln'].loc[model,metric],4) < round(files['global']['classic'].loc[model,metric],4) else 0
                            body_compC_p1[model][metric][indice]["e"] += 1 if round(files['personalized']['classic_mln'].loc[model,metric],4) == round(files['global']['classic'].loc[model,metric],4) else 0
                            body_compC_p1[model][metric][indice]["i"] += 1 if round(files['personalized']['classic_mln'].loc[model,metric],4) > round(files['global']['classic'].loc[model,metric],4) else 0

                            if sum([partern in clusters_p2[model][metric]['Good'] for partern in case_k.split("±")]) == k:
                               indice = 0
                            elif sum([partern in clusters_p2[model][metric]['Bad'] for partern in case_k.split("±")]) == k:
                               indice = 2
                            else:
                               indice = 1
                                
                            body_compC_p2[model][metric][indice]["s"] += 1 if round(files['personalized']['classic_mln_-_mlna'].loc[model,metric],4) < round(files['global']['classic'].loc[model,metric],4) else 0
                            body_compC_p2[model][metric][indice]["e"] += 1 if round(files['personalized']['classic_mln_-_mlna'].loc[model,metric],4) == round(files['global']['classic'].loc[model,metric],4) else 0
                            body_compC_p2[model][metric][indice]["i"] += 1 if round(files['personalized']['classic_mln_-_mlna'].loc[model,metric],4) > round(files['global']['classic'].loc[model,metric],4) else 0
                        dictio = {
                            'classic_-_mlna': 'Classic - Att',
                            'classic_mln': 'Classic + MLN',
                            'classic_mln_-_mlna': 'Classic + MLN - Att',
                            'classic': 'Classic'
                        }
                        for i, key in enumerate(files['global'].keys()): 
                            # find the best metric
                            temp = body_best[model][metric]["value"]
                            #print(f'model --> {model}  metric --> {metric}  value --> {temp}, {key}')
                            if round(files['global'][key].loc[model,metric],4) > temp:
                                body_best[model][metric]["value"] = round(files['global'][key].loc[model,metric],4)
                                body_best[model][metric]["approachs"] = [f'<span style={"color:blue;"}>{dictio[key]}</span>']
                                body_best[model][metric]["logics"] = ['global'] if key != "classic_-_mlna" else []
                            elif round(files['global'][key].loc[model,metric],4) == temp:
                                body_best[model][metric]["approachs"].append(dictio[key])
                                body_best[model][metric]["logics"].append('global') if key != "classic_-_mlna" else None
                            if round(files['personalized'][key].loc[model,metric],4) > temp:
                                body_best[model][metric]["value"] = round(files['personalized'][key].loc[model,metric],4)
                                body_best[model][metric]["approachs"] = [dictio[key]]
                                body_best[model][metric]["logics"] = ['personalized'] if key != "classic_-_mlna" else []
                            elif round(files['personalized'][key].loc[model,metric],4) == temp:
                                body_best[model][metric]["approachs"].append(dictio[key])
                                body_best[model][metric]["logics"].append('personalized') if key != "classic_-_mlna" else None

                            # count approachs impacts
                            if i > 0:
                                body_compA_g[model][metric][i-1]["s"] += 1 if round(files['global'][key].loc[model,metric],4) < round(files['global']['classic'].loc[model,metric],4) else 0
                                body_compA_g[model][metric][i-1]["e"] += 1 if round(files['global'][key].loc[model,metric],4) == round(files['global']['classic'].loc[model,metric],4) else 0
                                body_compA_g[model][metric][i-1]["i"] += 1 if round(files['global'][key].loc[model,metric],4) > round(files['global']['classic'].loc[model,metric],4) else 0

                                body_compA_p[model][metric][i-1]["s"] += 1 if round(files['personalized'][key].loc[model,metric],4) < round(files['personalized']['classic'].loc[model,metric],4) else 0
                                body_compA_p[model][metric][i-1]["e"] += 1 if round(files['personalized'][key].loc[model,metric],4) == round(files['personalized']['classic'].loc[model,metric],4) else 0
                                body_compA_p[model][metric][i-1]["i"] += 1 if round(files['personalized'][key].loc[model,metric],4) > round(files['personalized']['classic'].loc[model,metric],4) else 0
                                
                                
                        # count logics impacts
                        body_compL[model][metric][0]["s"] += 1 if round(files['personalized']['classic_mln'].loc[model,metric],4) > round(files['global']['classic_mln'].loc[model,metric],4) else 0
                        body_compL[model][metric][0]["e"] += 1 if round(files['personalized']['classic_mln'].loc[model,metric],4) == round(files['global']['classic_mln'].loc[model,metric],4) else 0
                        body_compL[model][metric][0]["i"] += 1 if round(files['personalized']['classic_mln'].loc[model,metric],4) < round(files['global']['classic_mln'].loc[model,metric],4) else 0
                        
                        body_compL[model][metric][1]["s"] += 1 if round(files['personalized']['classic_mln_-_mlna'].loc[model,metric],4) > round(files['global']['classic_mln_-_mlna'].loc[model,metric],4) else 0
                        body_compL[model][metric][1]["e"] += 1 if round(files['personalized']['classic_mln_-_mlna'].loc[model,metric],4) == round(files['global']['classic_mln_-_mlna'].loc[model,metric],4) else 0
                        body_compL[model][metric][1]["i"] += 1 if round(files['personalized']['classic_mln_-_mlna'].loc[model,metric],4) < round(files['global']['classic_mln_-_mlna'].loc[model,metric],4) else 0
   
            for metric in metrics:
                init = True
                for i, model in enumerate(models_name.keys()):
                    # approachs
                    #print(i,',')
                    tab2_body_g[metric] += (LayerLines + f'<td>{model}</td>'+ ''.join([
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["s"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compA_g[model][metric]])))}>{vector["s"]}</td>'+
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["e"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compA_g[model][metric]])))}>{vector["e"]}</td>'+
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["i"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compA_g[model][metric]])))}>{vector["i"]}</td>'  
                        for vector in body_compA_g[model][metric]
                        ]) + '</tr>') if init == True else (f'<tr><td>{model}</td>'+ ''.join([
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["s"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compA_g[model][metric]])))}>{vector["s"]}</td>'+
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["e"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compA_g[model][metric]])))}>{vector["e"]}</td>'+
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["i"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compA_g[model][metric]])))}>{vector["i"]}</td>'  
                        for vector in body_compA_g[model][metric]
                        ]) + '</tr>')
                    tab2_body_p[metric] += (LayerLines + f'<td>{model}</td>'+ ''.join([
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["s"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compA_p[model][metric]])))}>{vector["s"]}</td>'+
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["e"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compA_p[model][metric]])))}>{vector["e"]}</td>'+
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["i"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compA_p[model][metric]])))}>{vector["i"]}</td>'  
                        for vector in body_compA_p[model][metric]
                        ]) + '</tr>') if init == True else (f'<tr><td>{model}</td>'+ ''.join([
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["s"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compA_p[model][metric]])))}>{vector["s"]}</td>'+
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["e"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compA_p[model][metric]])))}>{vector["e"]}</td>'+
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["i"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compA_p[model][metric]])))}>{vector["i"]}</td>'  
                        for vector in body_compA_p[model][metric]
                        ]) + '</tr>')
                    init = False
                    # logics
                    tab4_body[metric] += (LayerLines + f'<td>{model}</td>'+ ''.join([
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["s"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compL[model][metric]])))}>{vector["s"]}</td>'+
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["e"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compL[model][metric]])))}>{vector["e"]}</td>'+
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["i"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compL[model][metric]])))}>{vector["i"]}</td>'  
                        for vector in body_compL[model][metric]
                        ]) + '</tr>') if i == 0 else (f'<tr><td>{model}</td>'+ ''.join([
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["s"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compL[model][metric]])))}>{vector["s"]}</td>'+
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["e"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compL[model][metric]])))}>{vector["e"]}</td>'+
                        f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["i"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compL[model][metric]])))}>{vector["i"]}</td>'  
                        for vector in body_compL[model][metric]
                        ]) + '</tr>')
                    # comparaison
                    if k == 2:
                        tab3_body_g1[metric] += (LayerLines + f'<td>{model}</td>'+ ''.join([
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["s"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_g1[model][metric]])))}>{vector["s"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["e"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_g1[model][metric]])))}>{vector["e"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["i"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_g1[model][metric]])))}>{vector["i"]}</td>'  
                            for vector in body_compC_g1[model][metric]
                            ]) + '</tr>') if i == 0 else (f'<tr><td>{model}</td>'+ ''.join([
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["s"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_g1[model][metric]])))}>{vector["s"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["e"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_g1[model][metric]])))}>{vector["e"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["i"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_g1[model][metric]])))}>{vector["i"]}</td>'  
                            for vector in body_compC_g1[model][metric]
                            ]) + '</tr>')
                        tab3_body_g2[metric] += (LayerLines + f'<td>{model}</td>'+ ''.join([
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["s"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_g2[model][metric]])))}>{vector["s"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["e"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_g2[model][metric]])))}>{vector["e"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["i"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_g2[model][metric]])))}>{vector["i"]}</td>'  
                            for vector in body_compC_g2[model][metric]
                            ]) + '</tr>') if i == 0 else (f'<tr><td>{model}</td>'+ ''.join([
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["s"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_g2[model][metric]])))}>{vector["s"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["e"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_g2[model][metric]])))}>{vector["e"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["i"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_g2[model][metric]])))}>{vector["i"]}</td>'  
                            for vector in body_compC_g2[model][metric]
                            ]) + '</tr>')
                        tab3_body_p1[metric] += (LayerLines + f'<td>{model}</td>'+ ''.join([
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["s"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_p1[model][metric]])))}>{vector["s"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["e"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_p1[model][metric]])))}>{vector["e"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["i"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_p1[model][metric]])))}>{vector["i"]}</td>'  
                            for vector in body_compC_p1[model][metric]
                            ]) + '</tr>') if i == 0 else (f'<tr><td>{model}</td>'+ ''.join([
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["s"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_p1[model][metric]])))}>{vector["s"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["e"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_p1[model][metric]])))}>{vector["e"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["i"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_p1[model][metric]])))}>{vector["i"]}</td>'  
                            for vector in body_compC_p1[model][metric]
                            ]) + '</tr>')
                        tab3_body_p2[metric] += (LayerLines + f'<td>{model}</td>'+ ''.join([
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["s"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_p2[model][metric]])))}>{vector["s"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["e"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_p2[model][metric]])))}>{vector["e"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["i"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_p2[model][metric]])))}>{vector["i"]}</td>'  
                            for vector in body_compC_p2[model][metric]
                            ]) + '</tr>') if i == 0 else (f'<tr><td>{model}</td>'+ ''.join([
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["s"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_p2[model][metric]])))}>{vector["s"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["e"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_p2[model][metric]])))}>{vector["e"]}</td>'+
                            f'<td style={"color:black; font-size: 50px; font-weight: bold;" * int(vector["i"] >= max(max([(ap["s"],ap["e"],ap["i"]) for ap in body_compC_p2[model][metric]])))}>{vector["i"]}</td>'  
                            for vector in body_compC_p2[model][metric]
                            ]) + '</tr>')
                    
        
        # approachs
        #1
        tabs = ''
        for metric in metrics:
            tabs += f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab2_head_g[metric]}{tab2_body_g[metric]}</table>'
      
        
        caption = f'<caption><h2>Legend</h2>{caption_content_lambda(metric)}</caption>'
        #table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tabs}</table>'
        htm = f'<html><head>{style}<title> Approachs comparaison on {data_folder} in Global logic </title></head><body style="background-color: white;">{caption}{tabs}</body></html>'
        
        create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab2/global')
        timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
        filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab2/global/Statistical comparaison of approachs in {data_folder} on global logic'+'_'+timestr+'.html'
        _file= open(filename1,"w")
        _file.write(htm)
        _file.close()
        #2
        tabs = ''
        for metric in metrics:
            tabs += f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab2_head_p[metric]}{tab2_body_p[metric]}</table>'
        
        caption = f'<caption><h2>Legend</h2>{caption_content_lambda(metric)}</caption>'
        #table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab2_head_p[metric]}{tab2_body_p[metric]}</table>'
        htm = f'<html><head>{style}<title> Approachs comparaison on {data_folder} in Personalized logic </title></head><body style="background-color: white;">{caption}{tabs}</body></html>'
        
        create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab2/personalized')
        timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
        filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab2/personalized/Statistical comparaison of approachs in {data_folder} on personalized logic'+'_'+timestr+'.html'
        _file= open(filename1,"w")
        _file.write(htm)
        _file.close()

        # logics
        tabs = ''
        for metric in metrics:
            tabs += f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab4_head[metric]}{tab4_body[metric]}</table>'
        
        caption = f'<caption><h2>Legend</h2>{caption_content_lambda(metric)}</caption>'
        #table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab4_head[metric]}{tab4_body[metric]}</table>'
        htm = f'<html><head>{style}<title> Comparaison of Logic for on {data_folder} </title></head><body style="background-color: white;">{caption}{tabs}</body></html>'
        
        create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab4')
        timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
        filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab4/Statistical comparaison of logics in {data_folder}'+'_'+timestr+'.html'
        _file= open(filename1,"w")
        _file.write(htm)
        _file.close()

        # comparaison
        #1
        tabs = ''
        for metric in metrics:
            tabs += f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab3_head_g[metric]}{tab3_body_g1[metric]}</table>'
        
        caption = f'<caption><h2>Legend</h2>{caption_content_lambda(metric)}</caption>'
        #table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab3_head_g[metric]}{tab3_body_g1[metric]}</table>'
        htm = f'<html><head>{style}<title> Tuple qualitity comparaison on {data_folder} based on Classic + MLN Approach and Global Logic </title></head><body style="background-color: white;">{caption}{tabs}</body></html>'
        
        create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab3/global')
        timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
        filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab3/global/Statistical comparaison of Tuple of MLN1 quality in {data_folder} on global logic using Classic + MLN apporach '+'_'+timestr+'.html'
        _file= open(filename1,"w")
        _file.write(htm)
        _file.close()
        #2
        tabs = ''
        for metric in metrics:
            tabs += f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab3_head_g[metric]}{tab3_body_g2[metric]}</table>'
        
        caption = f'<caption><h2>Legend</h2>{caption_content_lambda(metric)}</caption>'
        #table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab3_head_g[metric]}{tab3_body_g2[metric]}</table>'
        htm = f'<html><head>{style}<title> Tuple qualitity comparaison on {data_folder} based on Classic + MLN - Att Approach and Global Logic </title></head><body style="background-color: white;">{caption}{tabs}</body></html>'
        
        create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab3/global')
        timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
        filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab3/global/Statistical comparaison of Tuple of MLN1 quality in {data_folder} on global logic using Classic + MLN - Att apporach'+'_'+timestr+'.html'
        _file= open(filename1,"w")
        _file.write(htm)
        _file.close()
        #3
        tabs = ''
        for metric in metrics:
            tabs += f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab3_head_g[metric]}{tab3_body_p1[metric]}</table>'
        
        caption = f'<caption><h2>Legend</h2>{caption_content_lambda(metric)}</caption>'
        #table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab3_head_g[metric]}{tab3_body_p1[metric]}</table>'
        htm = f'<html><head>{style}<title> Tuple qualitity comparaison on {data_folder} based on Classic + MLN  and Personalized Logic </title></head><body style="background-color: white;">{caption}{tabs}</body></html>'
        
        create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab3/personalized')
        timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
        filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab3/personalized/Statistical comparaison of Tuple of MLN1 quality in {data_folder} on personalized logic using Classic + MLN apporach'+'_'+timestr+'.html'
        _file= open(filename1,"w")
        _file.write(htm)
        _file.close()
        #4
        tabs = ''
        for metric in metrics:
            tabs += f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab3_head_g[metric]}{tab3_body_p2[metric]}</table>'
        
        caption = f'<caption><h2>Legend</h2>{caption_content_lambda(metric)}</caption>'
        #table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab3_head_g[metric]}{tab3_body_p2[metric]}</table>'
        htm = f'<html><head>{style}<title> Tuple qualitity comparaison on {data_folder} based on Classic + MLN - Att Approach and Personalized Logic </title></head><body style="background-color: white;">{caption}{tabs}</body></html>'
        
        create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab3/personalized')
        timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
        filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab3/personalized/Statistical comparaison of Tuple of MLN1 quality in {data_folder} on personalized logic using Classic + MLN - Att apporach'+'_'+timestr+'.html'
        _file= open(filename1,"w")
        _file.write(htm)
        _file.close()
        
            # best metrics
        for metric in metrics:
            #max_bm = max([body_best[key][metric]["value"] for key in models_name.keys()])
            #max_bm_up = max([round(((body_best[key][metric]["value"] - round(files["global"]["classic"].loc[key,metric],1))/round(files["global"]["classic"].loc[key,metric],1))*100,4) for key in models_name.keys()])
            #print(body_best)
            for i, model in enumerate(models_name.keys()):
                tab1_body[metric] += (f'<tr style="border-top: 2px solid black;"><td rowspan="{len(models_name.keys()) }" >MLN k layer(s)</td>' + f'<td>{model}</td>'+ 
                    f'<td>{round(files["global"]["classic"].loc[model,metric],4)}</td>'+
                    f'<td >{body_best[model][metric]["value"]}</td>'+
                    f'<td >{round(((body_best[model][metric]["value"] - round(files["global"]["classic"].loc[model,metric],4))/round(files["global"]["classic"].loc[model,metric],4))*100,1)}</td>'+
                    f'<td style="overflow-wrap: break-word;word-wrap: break-word;">{" , ".join(list(set(body_best[model][metric]["approachs"])))}</td>'+
                    f'<td style="overflow-wrap: break-word;word-wrap: break-word;">{" , ".join(list(set(body_best[model][metric]["logics"])))}</td>'+  
                    '</tr>') if i == 0 else (f'<tr><td>{model}</td>'+ 
                    f'<td>{round(files["global"]["classic"].loc[model,metric],4)}</td>'+
                    f'<td >{body_best[model][metric]["value"]}</td>'+
                    f'<td >{round(((body_best[model][metric]["value"] - round(files["global"]["classic"].loc[model,metric],4))/round(files["global"]["classic"].loc[model,metric],4))*100,1)}</td>'+
                    f'<td style="overflow-wrap: break-word;word-wrap: break-word;">{" , ".join(list(set(body_best[model][metric]["approachs"])))}</td>'+
                    f'<td style="overflow-wrap: break-word;word-wrap: break-word;">{" , ".join(list(set(body_best[model][metric]["logics"])))}</td>'+  
                    '</tr>')
                
        tabs = ''
        for metric in metrics:
            tabs += f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab1_head[metric]}{tab1_body[metric]}</table>'
        
        caption = f'<caption><h2>Legend</h2>{caption_content_lambda(metric)}</caption>'
        #table_html = f'<table style="border: 2px solid black; width: 100% !important; background-color: #FFFFFF; color:#000000;">{tab1_head[metric]}{tab1_body[metric]}</table>'
        htm = f'<html><head>{style}<title> Best metric comparaison </title></head><body style="background-color: white;">{caption}{tabs}</body></html>'
        
        create_domain(f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab1')
        timestr = time.strftime("%Y_%m_%d_%H_%M_%S")
        filename1 = f'{cwd}/analyze_{outputs_path.split("/")[-2]}_made_on_{day}H/{data_folder}/plots/stats/tab1/Statistical best metric comparaison in {data_folder}'+'_'+timestr+'.html'
        _file= open(filename1,"w")
        _file.write(htm)
        _file.close()

In [9]:
get_qualitative_from_cols = lambda x:(list(set([
    var.split("__")[1] for var in [
        coll 
        for coll in [
            col 
            for col in x 
                if not (
                    ('precision' in col ) 
                    or ('accuracy' in col ) 
                    or ('recall' in col) 
                    or ('f1-score' in col)
                )
            ] 
            if ("__" in coll)
        ]
    ]
)))

In [12]:
def analyzer_launcher(outputs_name=None, analytical_func=None):
    
    result_folders = [dirnames for _, dirnames, _ in os.walk(f'{os.getcwd()}/{outputs_name}')][0]
    for dataset_name in result_folders:
        print(dataset_name)
        classic_f = [
                        load_data_set_from_url(path=file,sep='\t', encoding='utf-8',index_col=0, na_values=None) 
                        for file in get_filenames(
                            root_dir=f'{os.getcwd()}/{outputs_name}/{dataset_name}/data_selection_storage', 
                            func=MLN_C, 
                            verbose=False
                            )
                        ][-1]
        quali_col = get_qualitative_from_cols(classic_f.columns.to_list())
        models_list = classic_f.index.values.tolist()
        models = model_desc()
        models_name = { key : models[key] for key in models.keys() if key in models_list}
        
        analytical_func(
            cols=quali_col, 
            outputs_path=f'{os.getcwd()}/{outputs_name}/{dataset_name}', 
            cwd=os.getcwd(), 
            data_folder=dataset_name, 
            classic_metrics=classic_f, 
            models_name=models_name
            )
        

In [13]:
#analyzer_launcher(outputs_name="outputs_19_02_2024", analytical_func=metrics_analyzer_reports)

In [14]:
#analyzer_launcher(outputs_name="outputs_06032024", analytical_func=metrics_analyzer_reports)

In [15]:
#analyzer_launcher(outputs_name="outputs_1", analytical_func=metrics_analyzer_reports)

In [16]:
#analyzer_launcher(outputs_name="outputs_19_02_2024", analytical_func=metrics_analyzer_statistics)

In [17]:
analyzer_launcher(outputs_name="outputs_16032024", analytical_func=metrics_analyzer_reports)

AER
selfemp
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
owner
['xgb', 'dtc', 'lrc', 'rfc', 'sv']
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_16032024_made_on_2024_04_08_15H/AER/plots/mlna_1/report' created successfully.
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_16032024_made_on_2024_04_08_15H/AER/plots/mlna_1/report' already exists.
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_16032024_made_on_2024_04_08_15H/AER/plots/mlna_1/report' already exists.
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_16032024_made_on_2024_04_08_15H/AER/plots/mlna_1/report' already exists.
Directory '/Users/djiemboutientcheuvictornico/Documents/M2_Thesis/M2_thesis/scripting/analyze_outputs_16032024_made_on_2024_04_08_15H/AER/plots/mlna_1/report' already exists.
selfemp±owner
['xgb', 'dtc', 'l

In [18]:
#analyzer_launcher(outputs_name="outputs_1", analytical_func=metrics_analyzer_statistics)

\item Construire un graphe multicouches avec toutes les variables qualitatives (sans sélectionner arbitrairement 2 comme les auteurs),
\item Appliquer un PageRank personnalisé pour chaque exemple pour lequel on doit faire une prédiction, contrairement aux auteurs qui applique un PageRank personnalisé global dont les résultats sont répercuté sur tous les exemples (En d'autre termes, personnaliser l'exécution du PageRank pour chaque exemple du jeu de données).

In [19]:
# comparer uniquement les mlns et le classic. À faire